In [8]:
from jetbot import Robot
from Adafruit_MotorHAT.Adafruit_PWM_Servo_Driver import PWM
import traitlets
from traitlets.config.configurable import SingletonConfigurable
from traitlets.config.configurable import Configurable

robot = Robot()

class Jetracer(Configurable):
    
    turning = traitlets.Float()
    speed = traitlets.Float()
    
    def __init__(self, *args, **kwargs):
        super(Jetracer, self).__init__(*args, **kwargs)
        self.pwm = PWM(0x60, debug=False, i2c=None, i2c_bus=1)
        self.pwm.setPWMFreq(50)
        self.pwm.setPWM(14, 0, int((90 * (2000 / 180) + 501) * 4096 / 20000))
    
    @traitlets.observe('turning')
    def _observe_turning(self, change):
        self.pwm.setPWM(14, 0, int((90*( change['new'] + 1) * (2000 / 180) + 501) * 4096 / 20000))
    
    @traitlets.observe('speed')
    def _observe_speed(self, change):
        robot.left_motor.value = -change['new']
        robot.right_motor.value = -change['new']
    
    def stop(self):
        robot.stop();

In [9]:
jetracer = Jetracer()

In [10]:
jetracer.turning = 0.2
jetracer.speed = 0.3

In [11]:
jetracer.turning = -0.2
jetracer.speed = 0.3

In [12]:
jetracer.turning = 0.2
jetracer.speed = -0.3

In [13]:
jetracer.turning = -0.2
jetracer.speed = -0.3

In [14]:
jetracer.stop()

In [15]:
import ipywidgets.widgets as widgets

controller = widgets.Controller(index=0)  # replace with index of your controller

display(controller)

Controller()

In [16]:
turning_link = traitlets.dlink((controller.axes[0], 'value'), (jetracer, 'turning'), transform=lambda x: -x)
speed_link = traitlets.dlink((controller.axes[1], 'value'), (jetracer, 'speed'), transform=lambda x: -x)